## Setup

In [2]:
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA

In [3]:
import warnings
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

# %config IPCompleter.use_jedi = False
%matplotlib inline

np.set_printoptions(precision=2, suppress=True)
plt.style.use("ggplot")


In [4]:
SEED = 2020_2024

np.random.seed(SEED)

## Loading CSVs

In [5]:
base_dir = "/wrkspc/datasets/soil_vnir/csv_data"
csv_files = glob.glob(f"{base_dir}/*.csv")

for file in csv_files:
    print(file)


## Loading Data

In [6]:
# pd.read_csv("/wrkspc/datasets/soil_vnir/csv_data\\Trace_elements.csv", encoding="unicode_escape").columns # contains Mn, Mo, Pb, Ni, Cr, Co
# pd.read_csv("/wrkspc/datasets/soil_vnir/csv_data\\Soluble_salts.csv", encoding="unicode_escape").columns # Contains Cl, NO3, HCO3, SO4
# pd.read_csv("/wrkspc/datasets/soil_vnir/csv_data\\Elemental_composition_soil.csv", encoding="unicode_escape").columns # Contains MgO MnO2 CaO MgO K2O Al2O3
# pd.read_csv("/wrkspc/datasets/soil_vnir/csv_data\\Chemical_properties.csv", encoding="unicode_escape").isna().sum() # Contains H2O, KCl, N, NA, KG, Ca


In [7]:
spectra_data = pd.read_csv("ASD Spectra.csv",
                           encoding="unicode_escape", dtype={"Batch_Labid": "string"})
spectra_data


,Batch_Labid,W350,W360,W370,W380,W390,W400,W410,W420,W430,...,W2410,W2420,W2430,W2440,W2450,W2460,W2470,W2480,W2490,W2500
0,FS15R_FS4076,0.087266,0.072293,0.067882,0.071278,0.070909,0.075602,0.078873,0.082193,0.088196,...,0.394691,0.388342,0.378323,0.363642,0.355189,0.350998,0.347304,0.339293,0.336794,0.342761
1,FS15R_FS4077,0.091761,0.070821,0.069020,0.070133,0.072216,0.076107,0.079003,0.083118,0.089067,...,0.372140,0.364687,0.353332,0.338953,0.333400,0.325874,0.324145,0.326207,0.326426,0.330888
2,FS15R_FS4078,0.089091,0.069347,0.069658,0.068196,0.070051,0.071531,0.072116,0.075222,0.080212,...,0.317970,0.311350,0.300105,0.286625,0.280191,0.276309,0.276022,0.268724,0.279947,0.275649
3,FS15R_FS4079,0.094945,0.088997,0.081051,0.083512,0.085434,0.086396,0.087520,0.090224,0.095810,...,0.325006,0.316872,0.307036,0.291386,0.285994,0.280008,0.281164,0.280671,0.287855,0.290292
4,FS15R_FS4080,0.091239,0.065712,0.065946,0.066217,0.066276,0.064272,0.063520,0.065252,0.068709,...,0.369781,0.361111,0.348867,0.333307,0.325529,0.323597,0.323335,0.320587,0.325964,0.324467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4434,FS21_FS9996,0.047814,0.048310,0.046190,0.047078,0.046512,0.045272,0.046759,0.046725,0.048166,...,0.413332,0.407350,0.400803,0.393383,0.387425,0.383466,0.381514,0.379713,0.367057,0.379223
4435,FS21_FS9997,0.059975,0.041465,0.041414,0.042063,0.042857,0.041145,0.043146,0.043345,0.045756,...,0.362175,0.356727,0.348739,0.339670,0.334909,0.331045,0.328659,0.322995,0.319353,0.326614
4436,FS21_FS9998,0.073708,0.060482,0.052420,0.051500,0.051700,0.052734,0.052165,0.052380,0.054418,...,0.388367,0.382566,0.373157,0.363362,0.358007,0.352936,0.344330,0.341780,0.348666,0.344181
4437,FS21_FS9999,0.066636,0.058891,0.056509,0.052744,0.049982,0.050144,0.049327,0.049842,0.051371,...,0.386422,0.379406,0.370082,0.360227,0.354623,0.351410,0.345886,0.347730,0.337730,0.344610


In [8]:
glue = pd.read_csv(
    "ICRAF sample codes.csv", encoding="unicode_escape", dtype={"Batch and labid": "string", "Sampleno": "string"})
glue.columns = ["Batch_Labid", "SAMPLENO", "Country name",
                "Plotcode", "HORI", "BTOP", "BBOT", "Dsed"]
glue = glue.drop("Country name", axis=1).dropna()

glue["ISO"] = glue["Plotcode"].apply(lambda x: str(x).split(" ")[0])
glue["ID"] = glue["Plotcode"].apply(lambda x: str(x).split(" ")[1])

glue["ISO"] = glue["ISO"].astype("string")
glue["ID"] = glue["ID"].astype("int64")

glue = glue.drop("Plotcode", axis=1)
glue


,Batch_Labid,SAMPLENO,HORI,BTOP,BBOT,Dsed,ISO,ID
0,FS15R_FS4068,850157,1.0,0.0,14.0,4.822,BR,1
1,FS15R_FS4069,850158,2.0,14.0,30.0,26.805,BR,1
2,FS15R_FS4070,850159,3.0,30.0,38.0,31.503,BR,1
3,FS15R_FS4071,850160,4.0,38.0,50.0,40.451,BR,1
4,FS15R_FS4072,850161,5.0,50.0,80.0,49.071,BR,1
...,...,...,...,...,...,...,...,...
4434,FS21_FS9995,770062,10.0,120.0,140.0,1.849,ZM,1
4435,FS21_FS9996,770063,1.0,0.0,7.0,0.745,ZM,2
4436,FS21_FS9997,770064,2.0,7.0,20.0,1.775,ZM,2
4437,FS21_FS9998,770065,3.0,20.0,40.0,9.158,ZM,2


In [9]:
keys = ["SAMPLENO", "HORI", "BTOP", "BBOT", "ISO", "ID"]

glue.dtypes

Batch_Labid     string
SAMPLENO        string
HORI           float64
BTOP           float64
BBOT           float64
Dsed           float64
ISO             string
ID               int64
dtype: object

In [10]:
chemical_properties = pd.read_csv(
    "Chemical_properties.csv", encoding="unicode_escape")
chemical_properties.dropna(subset=keys, axis=0, inplace=True)

chemical_properties["SAMPLENO"] = chemical_properties["SAMPLENO"].astype("string")
chemical_properties["ISO"] = chemical_properties["ISO"].astype("string")
chemical_properties["HORI"] = chemical_properties["HORI"].astype("float64")


chemical_properties.iloc[:, :6].dtypes


ISO          string
ID            int64
HORI        float64
BTOP        float64
BBOT        float64
SAMPLENO     string
dtype: object

In [11]:
physical_properties = pd.read_csv(
    "Physical_properties.csv", encoding="unicode_escape")
physical_properties.dropna(subset=keys, axis=0, inplace=True)

physical_properties["SAMPLENO"] = physical_properties["SAMPLENO"].astype(
    "string")
physical_properties["ISO"] = physical_properties["ISO"].astype("string")
physical_properties["HORI"] = physical_properties["HORI"].astype("float64")


physical_properties.iloc[:, :6].dtypes

ISO          string
ID            int64
HORI        float64
BTOP        float64
BBOT        float64
SAMPLENO     string
dtype: object

In [12]:
print(spectra_data.shape)
print(glue.shape)
data = pd.merge(glue, spectra_data, on="Batch_Labid")
data


(4439, 217)
(4296, 8)


,Batch_Labid,SAMPLENO,HORI,BTOP,BBOT,Dsed,ISO,ID,W350,W360,...,W2410,W2420,W2430,W2440,W2450,W2460,W2470,W2480,W2490,W2500
0,FS15R_FS4068,850157,1.0,0.0,14.0,4.822,BR,1,0.108498,0.090940,...,0.429692,0.423609,0.413418,0.400037,0.393072,0.388640,0.384570,0.382684,0.376993,0.385871
1,FS15R_FS4069,850158,2.0,14.0,30.0,26.805,BR,1,0.114126,0.093812,...,0.392292,0.384562,0.373455,0.360353,0.351368,0.350279,0.345405,0.345660,0.342089,0.342892
2,FS15R_FS4070,850159,3.0,30.0,38.0,31.503,BR,1,0.114031,0.099315,...,0.381182,0.374040,0.360500,0.349091,0.341278,0.336983,0.334355,0.334629,0.331888,0.342487
3,FS15R_FS4071,850160,4.0,38.0,50.0,40.451,BR,1,0.090507,0.067269,...,0.318364,0.311986,0.300331,0.284864,0.277884,0.276207,0.274083,0.273752,0.275815,0.274547
4,FS15R_FS4072,850161,5.0,50.0,80.0,49.071,BR,1,0.086999,0.080636,...,0.313172,0.306682,0.294691,0.282718,0.276343,0.276402,0.273915,0.276475,0.271576,0.284989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4292,FS21_FS9995,770062,10.0,120.0,140.0,1.849,ZM,1,0.037664,0.043604,...,0.524432,0.518388,0.508907,0.500189,0.491928,0.486609,0.482752,0.482451,0.477877,0.483445
4293,FS21_FS9996,770063,1.0,0.0,7.0,0.745,ZM,2,0.047814,0.048310,...,0.413332,0.407350,0.400803,0.393383,0.387425,0.383466,0.381514,0.379713,0.367057,0.379223
4294,FS21_FS9997,770064,2.0,7.0,20.0,1.775,ZM,2,0.059975,0.041465,...,0.362175,0.356727,0.348739,0.339670,0.334909,0.331045,0.328659,0.322995,0.319353,0.326614
4295,FS21_FS9998,770065,3.0,20.0,40.0,9.158,ZM,2,0.073708,0.060482,...,0.388367,0.382566,0.373157,0.363362,0.358007,0.352936,0.344330,0.341780,0.348666,0.344181


In [13]:
data = pd.merge(data, chemical_properties, on=keys)
data = pd.merge(data, physical_properties, on=keys)
data

,Batch_Labid,SAMPLENO,HORI,BTOP,BBOT,Dsed,ISO,ID,W350,W360,...,PF27,PF34,PF42,COLE,SSA,PHYSREM,EDITDATE_y,VERIFIED_y,CODETYPE,TYPE
0,FS15R_FS4068,850157,1.0,0.0,14.0,4.822,BR,1,0.108498,0.090940,...,NaN,NaN,NaN,NaN,NaN,NaN,3/2/1995 0:00,NaN,BR0011,1
1,FS15R_FS4069,850158,2.0,14.0,30.0,26.805,BR,1,0.114126,0.093812,...,18.200001,14.800000,13.6,NaN,NaN,NaN,3/2/1995 0:00,NaN,BR0012,2
2,FS15R_FS4070,850159,3.0,30.0,38.0,31.503,BR,1,0.114031,0.099315,...,NaN,NaN,NaN,NaN,NaN,NaN,3/2/1995 0:00,NaN,BR0013,3
3,FS15R_FS4071,850160,4.0,38.0,50.0,40.451,BR,1,0.090507,0.067269,...,NaN,NaN,NaN,NaN,NaN,NaN,3/2/1995 0:00,NaN,BR0014,4
4,FS15R_FS4072,850161,5.0,50.0,80.0,49.071,BR,1,0.086999,0.080636,...,31.600000,30.299999,27.4,NaN,NaN,NaN,3/2/1995 0:00,NaN,BR0015,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173,FS21_FS9995,770062,10.0,120.0,140.0,1.849,ZM,1,0.037664,0.043604,...,NaN,NaN,NaN,NaN,NaN,NaN,12/27/1994 0:00,NaN,ZM001a,a
4174,FS21_FS9996,770063,1.0,0.0,7.0,0.745,ZM,2,0.047814,0.048310,...,NaN,NaN,NaN,NaN,NaN,NaN,12/27/1994 0:00,NaN,ZM0021,1
4175,FS21_FS9997,770064,2.0,7.0,20.0,1.775,ZM,2,0.059975,0.041465,...,NaN,NaN,NaN,NaN,NaN,NaN,12/27/1994 0:00,NaN,ZM0022,2
4176,FS21_FS9998,770065,3.0,20.0,40.0,9.158,ZM,2,0.073708,0.060482,...,NaN,NaN,NaN,NaN,NaN,NaN,12/27/1994 0:00,NaN,ZM0023,3


In [14]:
print(chemical_properties.columns)
print(physical_properties.columns)

Index(['ISO', 'ID', 'HORI', 'BTOP', 'BBOT', 'SAMPLENO', 'PHH2O', 'PHKCL',
       'PHCACL2', 'CACO3', 'CASO4', 'ORGC', 'ORGN', 'C/N', 'CA', 'MG', 'NA',
       'K', 'SUM', 'EXACID', 'EXAL', 'CECSOIL', 'CECCLAY', 'CECORG', 'ECEC',
       'BS', 'ALS', 'ESP', 'EC', 'CHEMREM', 'EDITDATE', 'VERIFIED'],
      dtype='object')
Index(['ISO', 'ID', 'HORI', 'BTOP', 'BBOT', 'SAMPLENO', 'GRAVEL', 'S1', 'S2',
       'S3', 'S4', 'S5', 'TSA', 'SI1', 'SI2', 'TSI', 'CLAY', 'DISPCL', 'BULK',
       'PF0', 'PF1', 'PF15', 'PF2', 'PF23', 'PF27', 'PF34', 'PF42', 'COLE',
       'SSA', 'PHYSREM', 'EDITDATE', 'VERIFIED', 'CODETYPE', 'TYPE'],
      dtype='object')


In [15]:
features = ["PHH2O", "PHKCL", "CA", "MG", "NA", "K", "CLAY", "ORGC"] #, "ORGN"]

In [16]:
# keys+["Batch_Labid"]+spectra_data.columns[1:].to_list()
X_columns = spectra_data.columns[1:].to_list()
y_columns = features  # keys+["Batch_Labid"]+features


In [17]:
data = data.loc[:, keys + ["Batch_Labid"] +
                features + spectra_data.columns[1:].to_list()]
data = data.dropna()

In [18]:
X_data = data.loc[:, X_columns].to_numpy()
y_data = data.loc[:, y_columns].to_numpy()

In [19]:
def sg_smooth(row): return savgol_filter(
    row, window_length=11, polyorder=6, deriv=1)


X_sg = np.array([sg_smooth(row) for row in X_data])

In [20]:
pca = PCA(n_components=20)
X_pca = pca.fit_transform(X_data)

print(pca.explained_variance_ratio_)
print(f"Singular Values: {pca.singular_values_}")
print(
    f"Total Variance Explained by all the components is {sum(pca.explained_variance_ratio_)}")
print()
print(f"Components:\n{pca.components_}")


[0.87 0.07 0.04 0.01 0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.  ]
Singular Values: [95.97 27.63 20.87  9.18  8.43  5.44  3.85  2.84  2.26  1.89  1.57  1.3
  0.99  0.92  0.75  0.68  0.54  0.48  0.43  0.4 ]
Total Variance Explained by all the components is 0.999880194626895

Components:
[[ 0.02  0.02  0.02 ...  0.05  0.05  0.05]
 [ 0.02  0.02  0.02 ... -0.11 -0.11 -0.1 ]
 [ 0.06  0.06  0.06 ...  0.12  0.12  0.11]
 ...
 [-0.15 -0.03 -0.01 ...  0.13  0.24  0.34]
 [-0.13 -0.05 -0.02 ...  0.01  0.04  0.16]
 [ 0.3   0.04 -0.01 ...  0.03  0.03 -0.13]]


In [21]:
np.savez("combined_data", X_data=X_data, X_sg=X_sg, y_data=y_data, X_pca = X_pca)